In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [3]:


colors_df = pd.DataFrame(data=[['red'],['blue'],['green'],['blue']], columns=['color'])
print('Before One Hot Encoding:')
display(colors_df)

one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(colors_df)

colors_df_encoded = one_hot_encoder.transform(colors_df)
colors_df_encoded = pd.DataFrame(data=colors_df_encoded, columns=one_hot_encoder.categories_)
print('\nAfter One Hot Encoding:')
display(colors_df_encoded)

Before One Hot Encoding:


,color
0,red
1,blue
2,green
3,blue



After One Hot Encoding:


,blue,green,red
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0


In [4]:
#  https://stackoverflow.com/questions/57518057/how-can-i-convert-an-image-from-pixels-to-one-hot-encodings

image_colors = [
    (0, 0, 0),  # black
    (127, 127, 127),  # grey
    (255, 127, 255),  # pink
]
path = 'ramp-data/metric_data/onehot_source/chips/OAM-309612-261633-19.tif'

In [5]:
import tensorflow as tf


In [10]:
# Create a "color reference" tensor from image_colors
color_reference = tf.cast(tf.constant(image_colors), dtype=tf.uint8)

# Load the image and obtain tensor with one-hot values
image = tf.io.decode_image(tf.io.read_file(path), channels=3)
comp = tf.equal(image[..., None, :], color_reference)
one_hot = tf.cast(tf.reduce_all(comp, axis=-1), dtype=tf.float32)

AttributeError: module 'tensorflow._api.v2.experimental' has no attribute 'decode_tiff'

In [12]:
from keras.utils.np_utils import to_categorical
categorical_labels = to_categorical(image_colors, num_classes=None)

In [13]:
categorical_labels

array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)

In [ ]:
# one more trial https://stackoverflow.com/questions/69942647/sparse-categorical-crossentropy-shape-problem-with-keras
num_class=None
def get_img_and_onehot_class(img_path, class):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels=3) # this function doens't exist for tiff files, only experimental
    """ Other preprocessing of image."""
    return img, tf.one_hot(class, num_class)

### Tryal with actual image

In [17]:
import matplotlib.pyplot as plt
from osgeo import gdal
import os
import numpy as np

In [18]:
# image_chip = gdal.Open(r'ramp-data/metric_data/onehot/chips/OAM-309612-261633-19.tif') # 3 bands
image_chip = gdal.Open(r'ramp-data/metric_data/onehot_source/chips/OAM-309606-261636-19.tif')

In [ ]:
categorical_chip = to_categorical(image_chip, num_classes=None)
# doesn't work

let's try convert the tif to png?

In [13]:
# %run python3 -m convert_tif_to_png --file ramp-data/metric_data/onehot_source/chips/OAM-309612-261633-19.tif
#  we can convert to png, but in our case it wouldn't make sense, as we lose the bands info

In [22]:
# https://stackoverflow.com/questions/68170720/tiff-workaround-for-tensorflow
import cv2
def parse_with_opencv(image_path):
    return cv2.imread(image_path.decode('UTF-8'))

img_path = [path]

ds = tf.data.Dataset.from_tensor_slices(img_path).map(
    lambda x: tf.numpy_function(parse_with_opencv, [x], Tout=tf.uint8)
)
# doesn't work:
# Unbatching a tensor is only supported for rank >= 1
# works after adding the [] to the image path, from here:
# https://stackoverflow.com/questions/55560620/valueerror-unbatching-a-tensor-is-only-supported-for-rank-1


In [ ]:
# https://stackoverflow.com/questions/67969346/how-to-create-tf-dataset-from-geotiff-files-using-custom-map-function
ds = tf.data.Dataset.from_tensor_slices([path])
ds = ds.map(lambda x: tf.py_function(parse_image, [x], [tf.float32])).batch(8)
# again Unbatching a tensor is only supported for rank >= 1
# yet doesn't work with the [], we can use the one above

changing all files in folder

In [ ]:
import glob

In [ ]:
folder_images = "ramp-data/metric_data/onehot_source/chips/"
img_list = [fille for fille in glob.glob(f'{folder_images}*.tif')]
output_folder = "ramp-data/metric_data/onehot/binarymasks/"

In [ ]:
img_list

In [ ]:
for img in img_list:
    print(img)
    cat_chip = to_categorical(img, num_classes=None)
    


In [ ]:
img_list

In [ ]:
cat_chip